In [11]:
import pandas as pd
import numpy as np 
import PIL
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from classes import *
import os
from Labeller import Labeller


In [3]:
img_dir = 'images/'
data_dir = 'data/'

In [4]:
n_pages_tag = re.compile(r'\?page=(?P<n_pages>\d{3,})')

In [4]:
def scrape(url , tag):
    r = requests.get(url)
    retval = re.findall(tag , r.text)
    if len(retval) > 1:
        return list(zip(*retval))
    elif len(retval) == 1:
        return retval[0]
    else:
        return None

In [19]:
n_pages = int(scrape('https://www.boat24.com/en/sailingboats/?page=&sort=datdesc' , n_pages_tag))
link_data = []
for page_number in tqdm(range(0 , n_pages , 20)):
    url = f'https://www.boat24.com/en/sailingboats/?page={page_number}&sort=datdesc'
    r = requests.get(url)
    soup = BeautifulSoup(r.text , 'html.parser')
    li_tags = soup.find_all('li')
    for li_tag in li_tags:
        links = li_tag.find_all('a')
        for link in links:
            href = link.get('href')
            title = link.get('title')
            text = link.text.strip()
            if text == 'Show offer':
                link_data.append({'href': href, 'title': title, 'text': text})
boat24_data_raw = pd.DataFrame(link_data)
boat24_data_raw.drop_duplicates(subset=['href'] , inplace=True)
boat24_data_raw.to_csv(data_dir + 'boat24_data_raw.csv' , index=False)

100%|██████████| 207/207 [02:38<00:00,  1.31it/s]


In [12]:
boat24_data_raw = pd.read_csv(data_dir + 'boat24_data_raw.csv')
sailboatdata_data = pd.read_csv(os.path.join(data_dir, "sailboatdata_data_hashed_cleaned.csv"))
boat_names = [Name_Classes[x] for x in sailboatdata_data['name'].unique()]
labeller = Labeller(boat_names)

In [13]:
for i in range(len(boat24_data_raw)):
    print(boat24_data_raw.iloc[i]['title'].lower().split(" "))
    found , label , dist = labeller.add_label(boat24_data_raw.iloc[i]['title'].lower().split(" "))
    print(Name_Classes[label] , dist)
    break


['grand', 'soleil', '', '46.3']


IndexError: list index out of range

In [6]:
# Average hash computation.
# See:
# https://www.hackerfactor.com/blog/index.php?/archives/432-Looks-Like-It.html
def average_hasher(image):
  # Read image.
  img = PIL.Image.open(image)

  # Simplify image by reducing its size and colors.
  pixels = img.convert("L").resize((8, 8), Image.ANTIALIAS).getdata()

  # Get the average pixel value.
  sum = 0.0
  for p in pixels: sum += p
  mean = sum / 64

  # Generate the hash by comparing each pixel's value to the average.
  bits = 0
  for p in pixels:
    bits <<= 1
    if p > mean: bits |= 1

  return "%016x" % bits, img.width, img.height